In [30]:
import requests
import pandas as pd
from datetime import datetime

def fetch_data(version):
    url = f"https://api.llama.fi/protocol/uniswap-{version}"
    response = requests.get(url)
    return response.json()

def fetch_volume_data():
    url = "https://api.llama.fi/summary/dexs/uniswap?excludeTotalDataChart=true&excludeTotalDataChartBreakdown=true&dataType=dailyVolume"
    response = requests.get(url)
    return response.json()

# Fetch TVL data for each version
v1_data = fetch_data('v1')
v2_data = fetch_data('v2')
v3_data = fetch_data('v3')

# Fetch volume data
volume_data = fetch_volume_data()

def process_tvl_data(data, version):
    rows = []
    for chain, chain_data in data['chainTvls'].items():
        if 'tvl' in chain_data:
            for entry in chain_data['tvl']:
                date = datetime.fromtimestamp(entry['date']).strftime('%Y-%m-%d')
                tvl = entry['totalLiquidityUSD']
                rows.append({
                    'date': date,
                    'chain': chain.lower(),  # Convert chain to lowercase
                    'version': f'UNISWAP V{version}',  # Standardize version format
                    'tvl': tvl
                })
    return rows

def process_volume_data(data):
    rows = []
    for entry in data['totalDataChartBreakdown']:
        date = datetime.fromtimestamp(entry[0]).strftime('%Y-%m-%d')
        for chain, versions in entry[1].items():
            for version, volume in versions.items():
                rows.append({
                    'date': date,
                    'chain': chain.lower(),  # Convert chain to lowercase
                    'version': version.upper(),  # Convert version to uppercase
                    'volume': volume
                })
    return rows

# Process TVL data
tvl_rows = (
    process_tvl_data(v1_data, 1) +
    process_tvl_data(v2_data, 2) +
    process_tvl_data(v3_data, 3)
)

# Process volume data
volume_rows = process_volume_data(volume_data)

# Create DataFrames
tvl_df = pd.DataFrame(tvl_rows)
volume_df = pd.DataFrame(volume_rows)

# Merge TVL and volume DataFrames
merged_df = pd.merge(tvl_df, volume_df, on=['date', 'chain', 'version'], how='outer')

# Sort the DataFrame
merged_df = merged_df.sort_values(['date', 'chain', 'version'])

# Fill NaN values with 0 or an appropriate value
merged_df['tvl'] = merged_df['tvl'].fillna(0)
merged_df['volume'] = merged_df['volume'].fillna(0)


In [31]:
merged_df.sample(20)

,date,chain,version,tvl,volume
11562,2024-04-14,polygon,UNISWAP V3,1.088955e+08,0.000000e+00
11514,2024-02-23,polygon,UNISWAP V3,9.191383e+07,0.000000e+00
6108,2024-07-03,filecoin,UNISWAP V3,2.374570e+06,0.000000e+00
5915,2024-07-04,boba,UNISWAP V3,1.585922e+06,0.000000e+00
5144,2024-05-14,rsk,UNISWAP V3,1.859984e+07,0.000000e+00
5825,2024-04-03,boba,UNISWAP V3,2.075008e+06,0.000000e+00
12422,2022-05-17,ethereum,UNISWAP V3,3.641192e+09,1.448106e+09
10153,2023-01-14,arbitrum,UNISWAP V3,8.443299e+07,5.734512e+07
5898,2024-06-15,boba,UNISWAP V3,1.702335e+06,0.000000e+00
9219,2024-07-02,celo,UNISWAP V3,2.278656e+07,0.000000e+00


In [33]:
# Write the DataFrame to a CSV file
merged_df.to_csv('csv_outputs/uniswap_tvl_volume_data.csv', index=False)

print(f"Data has been written to uniswap_tvl_volume_data.csv")
print(f"Total rows: {len(merged_df)}")
# print(merged_df.sample(20))

Data has been written to uniswap_tvl_volume_data.csv
Total rows: 14277
